In [1]:
! kubectl delete configmap docker-config 
! kubectl create configmap docker-config \
   --from-file=config.json

configmap "docker-config" deleted
configmap/docker-config created


In [2]:
from kubeflow import fairing
from kubeflow.fairing.kubernetes import utils as k8s_utils
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource
import glob

In [3]:
fairing.config.set_preprocessor(
    'python', 
    path_prefix="/", # 이 디렉토리로 복사해서 압축한다
    command=["python", "/train/mnist.py"],  # 생략하면 Job Pod 실행 안함 뭥미 Dockerfile의 CMD는 그냥 무시하남
    input_files = glob.glob("mnist/**", recursive=True),
    output_map = {
        "mnist": ""   # from: to
    }
)

context_source = MinioContextSource(
    endpoint_url='http://minio-service.kubeflow:9000',
    minio_secret='minio',
    minio_secret_key='minio123',
    region_name='fairing' # f"/kubeflow-{region_name}"으로 버킷 생성
)

fairing.config.set_builder(
    'cluster',
    registry = 'registry.kube-system.svc.cluster.local:30000',
    image_name = "my-first-cluster-build", 
    context_source = context_source,
    dockerfile_path="mnist/Dockerfile",
    push=True,
    cleanup=False, # default: False
)

fairing.constants.constants.KANIKO_IMAGE = \
    'gcr.io/kaniko-project/executor:latest' #
    #'gcr.io/kaniko-project/executor:v0.22.0' # 원래 값

fairing.config.set_deployer(
    'job',
    # job_name='my-python-file-fairing',
    # namespace='myspace', # default: 현재 네임스페이스
    # pod_spec_mutators=[
    #     k8s_utils.get_resource_mutator(cpu=1, memory=5)
    # ]
    cleanup=False, # default: True
)

fairing.config.run()

[I 210720 18:01:47 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7f771940aa20>
[I 210720 18:01:47 config:136] Using builder: <kubeflow.fairing.builders.cluster.cluster.ClusterBuilder object at 0x7f775a80d978>
[I 210720 18:01:47 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f771d99cd30>
[I 210720 18:01:47 cluster:46] Building image using cluster builder.
[I 210720 18:01:47 base:107] Creating docker context: /tmp/fairing_context_5s90b75n
[W 210720 18:01:47 manager:298] Waiting for fairing-builder-ww7zf-m26bp to start...
[W 210720 18:01:47 manager:298] Waiting for fairing-builder-ww7zf-m26bp to start...
[W 210720 18:01:47 manager:298] Waiting for fairing-builder-ww7zf-m26bp to start...
[I 210720 18:01:50 manager:304] Pod started running True


INFO[0000] GET KEYCHAIN
INFO[0000] running on kubernetes ....
E0720 18:01:49.581295       1 metadata.go:166] while reading 'google-dockercfg-url' metadata: http status code: 404 while fetching url http://metadata.google.internal./computeMetadata/v1/instance/attributes/google-dockercfg-url
INFO[0000] Retrieving image manifest tensorflow/tensorflow
INFO[0000] Retrieving image tensorflow/tensorflow from registry index.docker.io
INFO[0000] GET KEYCHAIN
INFO[0002] Built cross stage deps: map[]
INFO[0002] Retrieving image manifest tensorflow/tensorflow
INFO[0002] Returning cached image manifest
INFO[0002] Executing 0 build triggers
INFO[0002] Unpacking rootfs as cmd COPY /mnist/python-file-to-fairing.py      /train/mnist.py requires it.
INFO[0032] Taking snapshot of full filesystem...
INFO[0047] COPY /mnist/python-file-to-fairing.py      /train/mnist.py
INFO[0047] RUN  pip install -U pip
INFO[0047] cmd: /bin/sh
INFO[0047] args: [-c pip install -U pip]
INFO[0047] Running: [/bin/sh -c pip inst

[W 210720 18:02:44 job:101] The job fairing-job-b9d75 launched.
[W 210720 18:02:44 manager:298] Waiting for fairing-job-b9d75-ncjxj to start...
[W 210720 18:02:44 manager:298] Waiting for fairing-job-b9d75-ncjxj to start...
[W 210720 18:02:44 manager:298] Waiting for fairing-job-b9d75-ncjxj to start...
[I 210720 18:02:47 manager:304] Pod started running True


11493376/11490434 [==============================] - 0s 0us/step
2021-07-20 18:02:56.754385: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-20 18:02:57.990873: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-20 18:02:57.997589: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2200155000 Hz
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
flatten (Flatten)            (None, 784)               0
_________________________________________________________________
dense (Dense)                (None, 128)         

(<kubeflow.fairing.preprocessors.base.BasePreProcessor at 0x7f771940aa20>,
 <kubeflow.fairing.deployers.job.job.Job at 0x7f771d99cd30>)

# curl로 프라이빗 레지스트리 확인

In [4]:
! curl http://registry.kube-system.svc.cluster.local:30000/v2/_catalog

{"repositories":["my-first-cluster-build"]}


In [5]:
! curl http://registry.kube-system.svc.cluster.local:30000/v2/my-first-cluster-build/tags/list

{"name":"my-first-cluster-build","tags":["7ACF7483","1E1105C7","7B983025","F96A6D7A","8989BCB4","8E32619B","F2D98686","65B5E3B8","A4F51210","D5D54D3A","A8F8E4B1","31BD1854","F8872967","54A9E817","A7FF4276"]}


# Image 확인
- Host에서 실행
- fairing에서 난수로 생성한 tag를 변경해서 실행할 것 (예: E2683407)

### Image pull
```sh
docker pull registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```

### Image inspect
```sh
docker inspect registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```
Cmd 항목을 확인하고 faring 코드대로 image build 되었음을 확인
```json
...
            "Cmd": [
                "python",
                "/train/mnist.py"
            ],
...
```